In [4]:
import codecs
import numpy as np
import tensorflow as tf
from konlpy.tag import Twitter
from gensim.models import doc2vec

In [5]:
twitter = Twitter()

In [26]:
f = open('../embed/doc2vec_twitter_kowiki_300000_docs.model.word2vec_format', 'rt', encoding='utf-8')

In [27]:
line, vector_size = f.readline().strip().split(' ')

In [29]:
line = int(line)
vector_size = int(vector_size)

In [30]:
line, vector_size

(227267, 300)

In [31]:
wordvec = []
weights = []

In [32]:
for i in range(line):
    line = f.readline().strip().split(' ')
    wordvec.append(line[0])
    weights.append(line[1:])

In [40]:
result = 'wordvec = ' + str(wordvec) + '\n'
result = result + 'weights = ' + str(weights).replace("'", '') + '\n'

In [43]:
with open('./result.py', 'wt', encoding='utf-8') as f:
    f.write(result)

In [44]:
import result

In [45]:
len(result.wordvec)

227267

In [48]:
result.wordvec.index('asdf')

ValueError: 'asdf' is not in list

In [39]:
print(str(weights[:1]).replace("'", ''))

[[-1.8773185, 0.37166834, 1.4259578, -2.0417209, -0.43853414, -0.2836291, -0.26387936, -0.73732835, -0.20745742, -0.5073027, 0.17548813, 0.3922125, 1.5634485, -0.626405, -0.15602675, 0.5142029, 1.520502, 1.2895896, 0.7167296, -0.57740134, 0.6021382, 0.67277837, -1.5093353, 1.1352111, 0.9981327, 1.124528, 0.41686916, 1.6507996, 0.4231364, -0.70722777, 1.4517041, 0.58884126, -0.11467478, -1.5643542, -3.0304666, 2.6313894, -0.49167448, 1.0424012, -0.7976266, 3.2043667, -0.8840076, -1.0823847, 0.9213058, -1.4486398, -1.4286788, -1.1673855, -0.625236, -1.6572697, 0.700355, 1.4516041, -1.4464822, -0.89535844, 0.6874158, 0.10797035, 1.6845393, 2.0384052, 0.3262868, -0.4160993, -0.15948695, 1.1851761, 1.4670491, 1.3007796, 1.4325001, -0.11460957, -2.0998704, -0.58441144, -1.524216, 0.10555134, -2.4239376, 1.1791996, 0.556675, 0.4768787, -0.5349993, -1.2007997, -1.475621, 1.9118726, -0.8039332, 0.72913593, 1.7328005, 1.2784797, -1.356862, -0.65373296, -0.039450087, 1.7281874, -0.5432901, 1.9094

In [ ]:
with open('../embed/doc2vec_twitter_kowiki_300000_docs.model.word2vec_format', 'rt', encoding='utf-8') as f:
    line, vector_size = f.realine().strip().split(' ')

In [6]:
model = doc2vec.Doc2Vec.load('../embed/doc2vec_model/doc2vec_twitter_kowiki_300000_docs.model')

In [7]:
import gensim.models

In [8]:
wvmodel = gensim.models.KeyedVectors.load_word2vec_format('..\embed\doc2vec_model\doc2vec_twitter_kowiki_300000_docs.model.word2vec_format')

In [9]:
len(wvmodel.vocab)

227267

In [10]:
len(model.wv.vocab)

227267

In [11]:
max_sent_len = 30
vocab_size = len(model.wv.vocab)
vector_size = model.vector_size

In [12]:
def tagger(sent):
    return [word for word, tag in twitter.pos(sent) if tag not in ['Punctuation', 'Unknown']]

In [13]:
def vectorize(model):
    def vec(sent):
        return model.wv[filter(lambda x: x in model.wv.vocab, tagger(sent))]
    
    return vec

vec = vectorize(model)

In [14]:
def padding(max_len):
    def pad(sent):
        if len(sent) > max_len:
            return sent[:max_len]
        else:
            return np.vstack((sent, np.zeros((max_len - sent.shape[0], sent.shape[1]))))
    
    return pad

In [15]:
with codecs.open('../sample_data/movie_review/train/train_data', encoding='utf-8') as f:
    data = f.readlines()
    data = map(vec, data)
    data = list(map(padding(max_sent_len), data))

In [16]:
with open('../sample_data/movie_review/train/train_label') as f:
    label = f.readlines()
    label = list(map(int, label))

In [17]:
plc_embed = tf.placeholder(tf.float32, (None, max_sent_len, vector_size))
plc_label = tf.placeholder(tf.float32, (None))
training = tf.placeholder(tf.bool)
keep_prob = tf.placeholder(tf.float32)

In [18]:
embed_size = 128
block_num = 16
highway_num = 4

In [19]:
prenet = tf.layers.dense(plc_embed, embed_size, activation=tf.nn.relu)
prenet = tf.nn.dropout(prenet, keep_prob=keep_prob)

prenet = tf.layers.dense(prenet, embed_size // 2, activation=tf.nn.sigmoid)
prenet = tf.nn.dropout(prenet, keep_prob=keep_prob)

In [20]:
block_num = 16

output = tf.layers.conv1d(prenet, embed_size // 2, 1, 1, padding='SAME')
for n in range(2, block_num + 1):
    tmp = tf.layers.conv1d(prenet, embed_size // 2, n, 1, padding='SAME')
    output = tf.concat((output, tmp), axis=-1)

output = tf.layers.batch_normalization(output, training=training)
output = tf.nn.relu(output)

In [21]:
pool = tf.layers.max_pooling1d(output, 2, 1, padding='SAME')

In [22]:
conv1 = tf.layers.conv1d(pool, embed_size // 2, 3, 1, padding='SAME')
conv1 = tf.layers.batch_normalization(conv1, training=training)
conv1 = tf.nn.relu(conv1)

conv2 = tf.layers.conv1d(conv1, embed_size // 2, 3, 1, padding='SAME')
conv2 = tf.layers.batch_normalization(conv2, training=training)

In [23]:
highway = conv2 + prenet
for i in range(highway_num):
    H = tf.layers.dense(highway, embed_size // 2, activation=tf.nn.relu)
    T = tf.layers.dense(highway, embed_size // 2, activation=tf.nn.sigmoid)
    highway = H * T + highway * (1.0 - T)

In [24]:
fw = tf.nn.rnn_cell.GRUCell(embed_size // 2)
bw = tf.nn.rnn_cell.GRUCell(embed_size // 2)
o, s = tf.nn.bidirectional_dynamic_rnn(fw, bw, tf.transpose(highway, (1, 0, 2)), dtype=tf.float32, time_major=True)
o = tf.transpose(tf.concat(o, axis=-1), (1, 0, 2))

In [25]:
attn = tf.layers.dense(o, max_sent_len) / (max_sent_len ** 0.5)
attn = tf.matmul(tf.nn.softmax(attn), o)

In [20]:
conv = tf.layers.conv1d(attn, 1, 1, 1, padding='SAME')
conv = tf.reshape(conv, (-1, max_sent_len))

In [27]:
tf.argmax(tf.layers.dense(conv, 10), axis=1)

<tf.Tensor 'ArgMax:0' shape=(?,) dtype=int64>

In [21]:

result = tf.reduce_mean(tf.reshape(conv, (-1, max_sent_len)), axis=1)

In [35]:
loss = tf.reduce_sum(tf.square(result - plc_label))
opt = tf.train.AdamOptimizer(0.9).minimize(loss)

In [36]:
class Batch(object):
    def __init__(self, x, y, batch_size):
        self.total_x = x
        self.total_y = y
        self.batch_size = batch_size

        self.iter_per_epoch = len(x) // batch_size
        self.epochs_completed = 0

        self._iter = 0

    def __call__(self):
        start = self._iter * self.batch_size
        end = (self._iter + 1) * self.batch_size

        batch_x = self.total_x[start:end]
        batch_y = self.total_y[start:end]

        self._iter += 1
        if self._iter == self.iter_per_epoch:
            self.epochs_completed += 1
            self._iter = 0

        return batch_x, batch_y

In [37]:
batch = Batch(data, label, 32)
summary = tf.summary.merge([
    tf.summary.scalar('loss', loss)
])

In [38]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [39]:
writer = tf.summary.FileWriter('./summary/movie1', sess.graph)

In [41]:
n_epoch = 1000
for epoch in range(n_epoch):
    for n in range(batch.iter_per_epoch):
        batch_x, batch_y = batch()
        _, s = sess.run([opt, summary], feed_dict={plc_embed: batch_x, plc_label: batch_y, training: True, keep_prob: 0.5})
        writer.add_summary(s, epoch * n_epoch + n)

KeyboardInterrupt: 